<a href="https://colab.research.google.com/github/hwangsae91/project_so/blob/master/exploration/221018/%5BExp_10%5D20221018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# google colab전용
from google.colab import drive
drive.mount("/content/drive")

# exploration 12번째 과제
@ 황한용(3기/쏘카)

## 라이브러리 선언

In [2]:
import nltk
nltk.download("stopwords")

import numpy as np
import os
import pandas as pd
import re
import requests
import tensorflow as tf

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from summa.summarizer import summarize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import AdditiveAttention, Concatenate, Dense, Embedding, Input, LSTM, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

ModuleNotFoundError: No module named 'nltk'

## 상수선언

In [ ]:
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/data/news" # 데이터 기본경로
DATA_PATH = BASE_PATH + "/news_headlines_more.csv" # 기사데이터

CONSTRUCTIONS = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                "you're": "you are", "you've": "you have"} #  텍스트 정규화 사전

SOS_TOKEN = "<sos>" # 문장 시작토큰
EOS_TOKEN = "<eos>" # 문장 끝 토큰

EMBEDDING_DIM = 128
HIDDEN_SIZE = 256

pad_seq_kwargs = {
    "value":None # 추후 추가예정
    , "padding":"post"
    , "maxlen":None # 추후 추가예정
}
TRAIN_TEST_SPLIT_KWARGS = {
    "test_size":0.08, "random_state":2022
}

HIDDEN_STATE_NUM = 64 # hidden state의 노드수

fit_kwargs = {
    "epochs":50 # epoch 횟수
    , "batch_size":HIDDEN_STATE_NUM
    , "callbacks":EarlyStopping(monitor="val_acc", patience=3)
    , "shuffle" : True # epoch당 셔플을 할지의 여부
    ,"validation_data": None # 추후 추가예정
    , "verbose":1
}

## 함수

In [1]:

def preprocess_sentence(df:pd.DataFrame, col_name:str, remove_stopwords:bool=True) -> None:
    """
    영어 문장의 내용을 전처리하는 함수\n
    처리전의 내용은 스크렙을 통해 가저온 기사로 가정하고 데이터를 처리한다.
    처리과정은 아래의 순서에 따른다.
    - 텍스트 소문자화
    - \<br>, \<a href \= ...\> 등의 html 태그 제거
    - 괄호로 닫힌 문자열 (...) 제거 Ex) my husband (and myself!) for => my husband for
    - 쌍따옴표(") 제거
    - 약어 정규화(`CONSTRUCTIONS`상수 참고)
    - 소유격 제거. Ex) roland's -> roland
    - 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    - m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah
    - 불용어 제거/미제거

    Examples
    ----------
    >>> df = pd.DataFrame(data=[
    >>> [
    >>>     '''Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, 
    >>>     was a Sr Systems Engineer at Infosys with almost 5 years of work experience.'''
    >>>     '''Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year.'''
    >>> ],[
    >>>     '''upGrad learner switches to career in ML & Al with 90% salary hike''',
    >>>     '''Delhi techie wins free food from Swiggy for one year on CRED'''
    >>> ]
    >>> ], columns=['text', 'summary'])
    >>> preprocess_sentence(df, 'text')
    >>> print(df['text'])
    
    Parameters
    ----------
    df : DataFrame
        영어로 써있는 문장이 들어가있는 데이터프레임
    col_name : str
        전처리할 컬럼의 이름
    remove_stopwords : bool, default = True
        불용어 삭제여부

    Returns
    -------
    None

    """
    stopwords_eng_dict = set(CONSTRUCTIONS.words('english'))
    df[col_name] = df[col_name].str.lower()
    df[col_name] = BeautifulSoup("\n".join(df[col_name].to_list()), "lxml").text.split("\n")
    df[col_name] = df[col_name].replace(r"\([^)]*\)", "", regex=True)
    df[col_name] = df[col_name].replace(r"'s\b","", regex=True)
    df[col_name] = [
                     " ".join([
                         CONSTRUCTIONS.get(w, w)
                         for w in l
                         ]) for l in df[col_name].str.split()
                    ]
    df[col_name] = df[col_name].replace("[^a-zA-Z]", " ", regex=True)
    df[col_name] = df[col_name].replace("[m]{2,}", "mm", regex=True)
    if remove_stopwords:
        df[col_name] = [
                        " ".join([
                                w for w in l
                                if (not w in stopwords_eng_dict
                                and len(w) > 1)
                            ]) for l in df[col_name].str.split()
                        ]
    else:
        df[col_name] = [
                        " ".join([
                                w for w in l
                                if len(w) > 1
                            ]) for l in df[col_name].str.split()
                        ]

def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s.split()) <= max_len):
        cnt = cnt + 1
  print(f"전체 샘플 중 길이가 {max_len} 이하인 샘플의 비율: {cnt / len(nested_list)}")

def decode_sequence(input_seq, tar_word_to_index, headlines_max_len, encoder_model, decoder_model):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_word_to_index[SOS_TOKEN]

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_word_to_index[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        if sampled_token != EOS_TOKEN:
            decoded_sentence += " " + sampled_token

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == EOS_TOKEN or len(decoded_sentence.split()) >= (headlines_max_len-1)):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq, src_index_to_word):
    temp=''
    for i in input_seq:
        if (i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq, tar_word_to_index, tar_index_to_word):
    temp=''
    for i in input_seq:
        if ((i!=0 and i!=tar_word_to_index['sostoken']) and i!=tar_word_to_index['eostoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp

NameError: name 'pd' is not defined

속도의 문제로 인하여 소스코드를 변경하였다.<br>
불용어 제거까지 적용시 늦어도 30초내외로 걸린다.<br>

## 메인

### 전처리

In [ ]:
data = pd.read_csv(DATA_PATH, encoding='iso-8859-1')
data = data[['text','headlines']] # 필요데이터 select
data.head()

필요한 기사와 요약만 출력한다.<br>
앞으로 기사는 `data['text']`, 요약은 `data['headlines']`로 언급하겠다.

In [ ]:
data.drop_duplicates(subset = ['text'], inplace=True)
data.dropna(axis=0, inplace=True)
data.reset_index(inplace=True)

기사의 중복값을 제거하며, `null`인 값을 가지고 있는 데이터를 row기준으로 삭제한다.<br>
삭제후에는 다시 인덱스번호를 부여하여 인덱스 안에 빈값이 없도록 한다.

In [ ]:
preprocess_sentence(data,'text')
preprocess_sentence(data,'headlines',False)
data.replace("", np.nan, inplace=True)
data.dropna(axis=0, inplace=True)

기사는 불용어 제거, 요약은 불용어 제거하지 않은 상태로 전처리를 하며<br>
전처리로 인하여 인하여 생긴 빈 문자열은 제거한다.

In [ ]:
text_max_len = 60
headlines_max_len = 8

text_len = [len(s.split()) for s in data['text']]
headlines_len = [len(s.split()) for s in data['headlines']]

print(f"텍스트의 최소 길이 : {np.min(text_len)}")
print(f"텍스트의 최대 길이 : {np.max(text_len)}")
print(f"텍스트의 평균 길이 : {np.mean(text_len)}")
print(f"요약의 최소 길이 : {np.min(headlines_len)}")
print(f"요약의 최대 길이 : {np.max(headlines_len)}")
print(f"요약의 평균 길이 : {np.mean(headlines_len)}")

plt.subplot(1,2,1)
plt.boxplot(text_len)
plt.title('text')
plt.subplot(1,2,2)
plt.boxplot(headlines_len)
plt.title('headlines')
plt.tight_layout()
plt.show()

plt.title('text')
plt.hist(text_len, bins = 40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

plt.title('headlines')
plt.hist(headlines_len, bins = 40)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

below_threshold_len(text_max_len, data['text'])
below_threshold_len(headlines_max_len,  data['headlines'])

기사와 요약의 최소, 최대, 평균 길이와 길이 분포를 시각화함.

In [ ]:
data = data[data['text'].apply(lambda x: len(x.split()) <= text_max_len)]
data = data[data['headlines'].apply(lambda x: len(x.split()) <= headlines_max_len)]

In [ ]:
data['decoder_input'] = SOS_TOKEN + " " + data['headlines']
data['decoder_target'] = data['headlines'] + " " +EOS_TOKEN

In [ ]:
encoder_input = np.array(data['text']) # 인코더의 입력
decoder_input = np.array(data['decoder_input']) # 디코더의 입력
decoder_target = np.array(data['decoder_target']) # 디코더의 레이블

In [ ]:
# index shuffle
news_max_len = encoder_input.shape[0]
news_valid_len = int(news_max_len * 0.2)

shuffle_idx  = np.arange(news_max_len)
np.random.shuffle(shuffle_idx)

encoder_input = encoder_input[shuffle_idx]
decoder_input = decoder_input[shuffle_idx]
decoder_target = decoder_target[shuffle_idx]

# 학습과 테스트, 점증 데이터 분리

encoder_input_train = encoder_input[:-news_valid_len]
decoder_input_train = decoder_input[:-news_valid_len]
decoder_target_train = decoder_target[:-news_valid_len]

encoder_input_test = encoder_input[-news_valid_len:]
decoder_input_test = decoder_input[-news_valid_len:]
decoder_target_test = decoder_target[-news_valid_len:]

In [ ]:
src_tokenizer = Tokenizer() # 토크나이저 정의
src_tokenizer.fit_on_texts(encoder_input_train) # 입력된 데이터로부터 단어 집합 생성

In [ ]:
threshold = 7
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print(f"단어 집합(vocabulary)의 크기 : {total_cnt}", )
print(f"등장 빈도가 {threshold - 1}번 이하인 희귀 단어의 수: {rare_cnt}")
print(f"단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 {total_cnt - rare_cnt}")
print(f"단어 집합에서 희귀 단어의 비율: {(rare_cnt / total_cnt)*100}")
print(f"전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {(rare_freq / total_freq)*100}")

In [ ]:
src_vocab = 8000
src_tokenizer = Tokenizer(num_words=src_vocab) # 단어 집합의 크기를 8,000으로 제한
src_tokenizer.fit_on_texts(encoder_input_train) # 단어 집합 재생성

# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train) 
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

# 잘 진행되었는지 샘플 출력
print(encoder_input_train[:3])

In [ ]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)

In [ ]:
threshold = 6
total_cnt = len(tar_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print(f"단어 집합(vocabulary)의 크기 : {total_cnt}", )
print(f"등장 빈도가 {threshold - 1}번 이하인 희귀 단어의 수: {rare_cnt}")
print(f"단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 {total_cnt - rare_cnt}")
print(f"단어 집합에서 희귀 단어의 비율: {(rare_cnt / total_cnt)*100}")
print(f"전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {(rare_freq / total_freq)*100}")

In [ ]:
tar_vocab = 2000
tar_tokenizer = Tokenizer(num_words=tar_vocab) 
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

# 텍스트 시퀀스를 정수 시퀀스로 변환
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train) 
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_target_test)

# 잘 변환되었는지 확인
print('input')
print('input ',decoder_input_train[:5])
print('target')
print('decoder ',decoder_target_train[:5])

In [ ]:
drop_train = set([index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1])
drop_test = set([index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1])

print(f"삭제할 훈련 데이터의 개수 : {len(drop_train)}")
print(f"삭제할 테스트 데이터의 개수 : {len(drop_test)}")

encoder_input_train = [sentence for index, sentence in enumerate(encoder_input_train) if index not in drop_train]
decoder_input_train = [sentence for index, sentence in enumerate(decoder_input_train) if index not in drop_train]
decoder_target_train = [sentence for index, sentence in enumerate(decoder_target_train) if index not in drop_train]

encoder_input_test = [sentence for index, sentence in enumerate(encoder_input_test) if index not in drop_test]
decoder_input_test = [sentence for index, sentence in enumerate(decoder_input_test) if index not in drop_test]
decoder_target_test = [sentence for index, sentence in enumerate(decoder_target_test) if index not in drop_test]

print(f"훈련 데이터의 개수 : {len(encoder_input_train)}")
print(f"훈련 레이블의 개수 : {len(decoder_input_train)}")
print(f"테스트 데이터의 개수 : {len(encoder_input_test)}")
print(f"테스트 레이블의 개수 : {len(decoder_input_test)}")

In [ ]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen=text_max_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen=text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen=headlines_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen=headlines_max_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen=headlines_max_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen=headlines_max_len, padding='post')

### 모델 설계

In [ ]:
# 인코더
encoder_inputs = Input(shape=(text_max_len,))

# 인코더의 임베딩 층
enc_emb = Embedding(src_vocab, EMBEDDING_DIM)(encoder_inputs)

# 인코더의 LSTM 1
# encoder_lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True ,dropout = 0.4, recurrent_dropout = 0.4)
encoder_lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True ,dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
# encoder_lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True, dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
# encoder_lstm3 = LSTM(HIDDEN_SIZE, return_state=True, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)
encoder_lstm3 = LSTM(HIDDEN_SIZE, return_state=True, return_sequences=True, dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

In [ ]:
# 디코더 설계
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb_layer = Embedding(tar_vocab, EMBEDDING_DIM)
dec_emb = dec_emb_layer(decoder_inputs)

# 디코더의 LSTM
# decoder_lstm = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
decoder_lstm = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True, dropout=0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

In [ ]:
# 어텐션 층(어텐션 함수)
attn_layer = AdditiveAttention(name='attention_layer')

# 인코더와 디코더의 모든 time step의 hidden state를 어텐션 층에 전달하고 결과를 리턴
attn_out = attn_layer([decoder_outputs, encoder_outputs])


# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

In [ ]:
fit_kwargs['validation_data'] = ([encoder_input_test, decoder_input_test], decoder_target_test)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005)
    , loss="sparse_categorical_crossentropy", metrics=['acc']
)
history = model.fit(
            x=[encoder_input_train, decoder_input_train]
            , y=decoder_target_train
            , **fit_kwargs).history

history_dict = model.fit(
                         x=[encoder_input_train, decoder_input_train]
                         , y=decoder_target_train
                         , **fit_kwargs).history

loss = history_dict['loss']
val_loss = history_dict['val_loss']

epoch = range(1, len(loss) + 1)

plt.plot(epoch, loss, 'bo', label='Training loss')
plt.plot(epoch, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

학습결과 우하향하는 그래프의 모습을 보이며 train loss, validation loss 의 차이가 크지않다.

### 검증


In [ ]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

In [ ]:
# 인코더 설계
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c = Input(shape=(HIDDEN_SIZE,))

dec_emb_infer_enc = dec_emb_layer(decoder_inputs)

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
(
    decoder_outputs_infer_enc
    , state_h_infer_enc
    , state_c_infer_enc
) = decoder_lstm(dec_emb_infer_enc, initial_state=[decoder_state_input_h, decoder_state_input_c])

In [ ]:
# 어텐션 함수
decoder_hidden_state_input = Input(shape=(text_max_len, HIDDEN_SIZE))
attn_out_inf = attn_layer([decoder_outputs_infer_enc, decoder_hidden_state_input])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs_infer_enc, attn_out_inf])

# 디코더의 출력층
decoder_outputs_infer_dec = decoder_softmax_layer(decoder_inf_concat)

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs_infer_dec] + [state_h2, state_c2])

print('=3')

In [ ]:
for i in range(50, 100):
    print("원문 :", seq2text(encoder_input_test[i]))
    print("실제 요약 :", seq2summary(decoder_input_test[i]))
    print("예측 요약 :", decode_sequence(encoder_input_test[i].reshape(1, text_max_len)))
    print("\n")

### 회고


- 학습된 데이터가 전체적으로 높은 정확도를 보여 미리 학습된 데이터의 중요성을 알게되었다.
- 항상 학습된 데이터가 높은 정확도를 나타내는 것이 아니라 하이퍼파라미터의 튜닝에 따라 결과가 달라질 수 있다는 것을 알게 되었다.

※ 이번 레포트는 양희성님의 모델구조의 조언으로 작성되었음을 알려드립니다.<br>
   희성님에게 감사하다는 글을 남기며 이만 글을 마치겠습니다.